In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

from sklearn.preprocessing import FunctionTransformer


In [2]:
df = pd.read_csv("Project 1\cybersecurity_attacks_new.csv")
df.head()

,Unnamed: 0,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,...,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source,os,os_version,browser,browser_version,device,device_model
0,0,2023-05-30 06:33:58,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,...,150.9.97.135,Log Data,NaN,Server,Windows,8,IE,9.0,NaN,NaN
1,1,2020-08-26 07:08:30,78.199.217.198,66.191.137.154,17245,48166,ICMP,1174,Data,HTTP,...,NaN,Log Data,NaN,Firewall,Windows,Vista,IE,8.0,NaN,NaN
2,2,2022-11-13 08:23:25,63.79.210.48,198.219.82.17,16811,53600,UDP,306,Control,HTTP,...,114.133.48.179,Log Data,Alert Data,Firewall,Windows,8,IE,9.0,NaN,NaN
3,3,2023-07-02 10:38:46,163.42.196.10,101.228.192.255,20018,32534,UDP,385,Data,HTTP,...,NaN,NaN,Alert Data,Firewall,Mac OS X,10.11.5,Firefox,11.0,Apple,Mac
4,4,2023-07-16 13:11:07,71.166.185.76,189.243.174.238,6131,26646,TCP,1462,Data,DNS,...,149.6.110.119,NaN,Alert Data,Firewall,Windows,8,IE,5.0,NaN,NaN


In [3]:
target = 'Attack Type'
print(df[target].value_counts())

Attack Type
DDoS         13428
Malware      13307
Intrusion    13265
Name: count, dtype: int64


In [9]:
df['Alerts/Warnings'] = df['Alerts/Warnings'].notna().astype(int)
df['IDS/IPS Alerts'] = df['IDS/IPS Alerts'].notna().astype(int)
df['Malware Indicators'] = df['Malware Indicators'].notna().astype(int)
df['Firewall Logs'] = df['Firewall Logs'].notna().astype(int)

to_drop_cols = ['Timestamp','Payload Data','User Information','Device Information', 'Source Port',
             'Geo-location Data','Proxy Information','Source IP Address','Destination IP Address','Packet Lenght']
# df_cleaned= df.drop(to_drop_cols, axis=1)
df_cleaned = df[[target,'Protocol', 'Traffic Type' ,'os','browser','Packet Type',
                 'Alerts/Warnings','Action Taken','Severity Level','Log Source',
                 'Anomaly Scores',
                 'Network Segment','device_model']]

categorical_cols = x.select_dtypes(include=['object']).columns
numerical_cols = x.select_dtypes(include=['int64', 'float64']).columns
print("Categorical Features:", categorical_cols)
x = df_cleaned.drop(target,axis=1)
y = df_cleaned[target]
print(x)

Categorical Features: Index(['Protocol', 'Traffic Type', 'os', 'browser', 'Packet Type',
       'Action Taken', 'Severity Level', 'Log Source', 'Network Segment',
       'device_model'],
      dtype='object')
      Protocol Traffic Type        os        browser Packet Type  \
0         ICMP         HTTP   Windows             IE        Data   
1         ICMP         HTTP   Windows             IE        Data   
2          UDP         HTTP   Windows             IE     Control   
3          UDP         HTTP  Mac OS X        Firefox        Data   
4          TCP          DNS   Windows             IE        Data   
...        ...          ...       ...            ...         ...   
39995      UDP         HTTP       iOS    Firefox iOS     Control   
39996      UDP         HTTP   Windows         Safari     Control   
39997      UDP          DNS   Windows         Safari        Data   
39998      UDP          FTP     Linux         Chrome        Data   
39999     ICMP         HTTP       iOS  Mobi

In [ ]:






# One hot encoding of categorical features
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(x)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree Classifier with pruning parameters
clf = DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on testing set: {accuracy}")

# Cross-validation
scores = cross_val_score(clf, X_encoded, y, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {scores.mean()}")

# Hyperparameter tuning using Grid Search
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}
grid_search = GridSearchCV(clf, param_grid, cv=5)
grid_search.fit(X_encoded, y)
print(f"Best parameters: {grid_search.best_params_}")

# Train the classifier with the best parameters
best_clf = grid_search.best_estimator_
best_clf.fit(X_train, y_train)

# Make predictions with the best model
best_y_pred = best_clf.predict(X_test)

# Calculate accuracy with the best model
best_accuracy = accuracy_score(y_test, best_y_pred)
print(f"Accuracy with best model on testing set: {best_accuracy}")

Accuracy on testing set: 0.330625
Cross-validation scores: [0.3355   0.335625 0.33625  0.335875 0.33575 ]
Mean cross-validation score: 0.3358
